In [32]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [33]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [34]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [35]:
if "Alarm" in cnc.status:
    cnc.home()

# Generate Program


In [38]:
grid_spacing = 2.5 # [mm]

In [39]:
Xs = np.arange(grid_spacing, BlockLength, grid_spacing)
Ys = np.arange(grid_spacing, BlockHeight, grid_spacing)

In [40]:
Xs=np.linspace(0, 1, 5)
Xs

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [41]:
Ys=np.linspace(0, 1, 6)
Ys

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [42]:
Xs=[0, 1, 2]
Ys=[4, 8]

In [43]:
(xx, yy) = np.meshgrid(Xs, Ys)

In [44]:
xx

array([[0, 1, 2],
       [0, 1, 2]])

In [45]:
yy

array([[4, 4, 4],
       [8, 8, 8]])

In [48]:
xx[0]

array([0, 1, 2])

In [49]:
xx[0][1]

1

In [50]:
yy[0]

array([4, 4, 4])

In [51]:
yy[1]

array([8, 8, 8])

In [52]:
xx[0][1], yy[0][1]

(1, 4)

In [53]:
xx[1][1], yy[0][2]

(1, 4)

In [55]:
len(Xs)

3

In [56]:
len(Ys)

2

In [59]:
for idx1, X in enumerate(Xs):
    for idx2, Y in enumerate(Ys):
        print("({}, {}): {} {}".format(X, Y, xx[idx1, idx2], yy[idx1, idx2]))

(0, 4): 0 4
(0, 8): 1 4
(1, 4): 0 8
(1, 8): 1 8


IndexError: index 2 is out of bounds for axis 0 with size 2

In [60]:
for idx1, X in enumerate(Xs):
    for idx2, Y in enumerate(Ys):
        print("({}, {}): {} {}".format(X, Y, xx[idx2, idx1], yy[idx2, idx1]))

(0, 4): 0 4
(0, 8): 0 8
(1, 4): 1 4
(1, 8): 1 8
(2, 4): 2 4
(2, 8): 2 8


In [64]:
Xs=[0, 10, 20]
Ys=[15, 30, 45]

XX, YY = np.meshgrid(Xs, Ys)

for idx1, X in enumerate(Xs):
    for idx2, Y in enumerate(Ys):
        assert (X == XX[idx2, idx1])
        assert (Y == YY[idx2, idx1])
        
        print("({}, {}) = ({}, {})".format(X, Y, XX[idx2, idx1], YY[idx2, idx1]))

(0, 15) = (0, 15)
(0, 30) = (0, 30)
(0, 45) = (0, 45)
(10, 15) = (10, 15)
(10, 30) = (10, 30)
(10, 45) = (10, 45)
(20, 15) = (20, 15)
(20, 30) = (20, 30)
(20, 45) = (20, 45)


In [31]:
for x, y in zip(xx, yy):
    for x_, y_ in zip(x, y):
        print(x_)
        print(y_)

0
4
1
4
2
4
0
8
1
8
2
8


In [29]:
powers = np.linspace(100, 255, len(Xs), dtype=np.uint8)
powers

array([100, 177, 255], dtype=uint8)

In [30]:
dwells = [1]

In [42]:
for dwell in dwells:
    break

In [43]:
cnc.init()

True

In [51]:
Zs = [-14]
for Z in Zs:
    break

In [53]:
programs=list()
def init2(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=500)
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    return program
for X, power in zip(Xs, powers):
    program = init2(machine=cnc)
    program.G0(Z=Z, Y=Y, X=X)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.
    programs.append(program)
program

<GCode>[cmds=9]

In [54]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="")
    print(cnc.run(program))

0: 4.641655206680298
1: 3.4909920692443848
2: 3.493727207183838
3: 3.494004964828491
4: 3.4909751415252686
5: 3.491802453994751
6: 3.490959405899048
7: 3.493772029876709
8: 3.491602897644043
9: 3.4925568103790283
10: 3.492128610610962
11: 3.491084337234497
12: 3.490161180496216
13: 3.4915528297424316
14: 3.491367816925049
15: 3.4912168979644775
16: 3.491985321044922
17: 3.493729591369629
18: 3.4935190677642822
19: 3.4953453540802
20: 3.492088794708252
21: 3.490997076034546
22: 3.4937238693237305
23: 3.4914145469665527
24: 3.491617202758789
25: 3.4921069145202637
26: 3.491852045059204
27: 3.4942235946655273


In [57]:
Y = Ys[1]
Y

10.0

In [58]:
programs=list()
dwell = 0.5
for X, power in zip(Xs, powers):
    program = init2(machine=cnc)
    program.G0(Z=Z, Y=Y, X=X)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.
    programs.append(program)
program

<GCode>[cmds=9]

In [59]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="")
    print(cnc.run(program))

0: 13.053906202316284
1: 2.8921163082122803
2: 2.8909823894500732
3: 2.8900773525238037
4: 2.890291690826416
5: 2.8894340991973877
6: 2.8901193141937256
7: 2.8906636238098145
8: 2.8908770084381104
9: 2.8906781673431396
10: 2.889885425567627
11: 2.8917291164398193
12: 2.8930020332336426
13: 2.8907077312469482
14: 2.890251874923706
15: 2.890632152557373
16: 2.890575647354126
17: 2.890132188796997
18: 2.8905398845672607
19: 2.8903121948242188
20: 2.8902411460876465
21: 2.891510009765625
22: 2.893519401550293
23: 2.8904337882995605
24: 2.88966703414917
25: 2.8901472091674805
26: 2.8902535438537598
27: 2.8902368545532227


In [60]:
dwell = 0.25
Y = Ys[2]
for X, power in zip(Xs, powers):
    program = init2(machine=cnc)
    program.G0(Z=Z, Y=Y, X=X)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.
    print(cnc.run(program))

13.054593563079834
2.891545057296753
2.8903682231903076
2.891618251800537
2.8935091495513916
2.8904051780700684
2.8903732299804688
2.890268087387085
2.8903684616088867
2.890045642852783
2.890733003616333
2.890333652496338
2.890665292739868
2.8916332721710205
2.890410900115967
2.891117811203003
2.8899118900299072
2.890746831893921
2.8902740478515625
2.8903446197509766
2.8945207595825195
2.8902759552001953
2.8907370567321777
2.891649007797241
2.8908002376556396
2.8906190395355225
2.8900022506713867
2.8908162117004395


In [ ]:
dwell = 2
Y = Ys[3]
for X, power in zip(Xs, powers):
    program = init2(machine=cnc)
    program.G0(Z=Z, Y=Y, X=X)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.
    print(cnc.run(program))

14.857101917266846
4.694027900695801
4.693021059036255
4.692686080932617
4.693039178848267
4.698350667953491
4.692682981491089
4.692937850952148
4.692770957946777
4.692894220352173
4.6926703453063965
4.692623853683472
4.693691968917847
4.692799091339111
4.693265438079834
4.6934754848480225
4.695194244384766
4.695768356323242
4.693379640579224
4.6930458545684814
4.692551374435425
4.6929967403411865
4.692992925643921
4.693429231643677
4.6929168701171875
4.693992853164673
4.695772171020508
4.693402528762817


In [ ]:
dwell = 5
Y = Ys[4]
for X, power in zip(Xs, powers):
    program = init2(machine=cnc)
    program.G0(Z=Z, Y=Y, X=X)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.
    print(cnc.run(program))

17.861169576644897
7.697419166564941
7.69661808013916
7.696374416351318
7.696740627288818
7.697359561920166
7.700284004211426
7.697939395904541
7.697470426559448
7.697144031524658
7.700500726699829
7.697020053863525
